<a href="https://colab.research.google.com/github/SheldonBurke/Math-for-Data-Science-Course/blob/main/Lil_Kim_Alexnet_Project_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from google.colab import drive
from torchsummary import summary

In [ ]:
# 1. Mount Google Drive
drive.mount('/content/gdrive')

# 2. Load labeled images from folders
# data_dir = '/content/gdrive/MyDrive/Data1/antsbeesdataset'
data_dir = '/content/gdrive/MyDrive/AI_Test_Kim_Project/Lil_Kim_Test'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/MyDrive/AI_Test_Kim_Project/Lil_Kim_Test

/content/gdrive/MyDrive/AI_Test_Kim_Project/Lil_Kim_Test


In [ ]:
ls # folders in the directory

alexnet_classification.pth  train/  valid/


In [ ]:
# 3. Pre-process the data and create data loaders
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
image_datasets = {x: datasets.ImageFolder(data_dir + '/' + x, data_transforms[x]) for x in ['train', 'valid']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset_sizes

{'train': 30, 'valid': 10}

In [ ]:
# 4. Set up the AlexNet architecture
alexnet = models.alexnet(pretrained=True)

In [ ]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:

num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names)) # Change last layer
alexnet = alexnet.to(device) # Put on GPU

In [ ]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
summary(alexnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [ ]:
class_names

['Kim Jong IL', 'Kim Jong UN']

In [ ]:
dataset_sizes

{'train': 30, 'valid': 10}

In [ ]:
for inputs, labels in dataloaders["train"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

In [ ]:
inputs.shape

torch.Size([14, 3, 224, 224])

In [ ]:
labels.shape

torch.Size([14])

In [ ]:
labels

tensor([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1], device='cuda:0')

In [ ]:
outputs = alexnet(inputs)

In [ ]:
outputs.shape

torch.Size([14, 2])

In [ ]:
outputs

tensor([[ 0.0378,  1.2452],
        [ 0.1660,  0.1548],
        [-0.0108,  0.0464],
        [ 0.5080,  0.7897],
        [ 0.5860,  0.0024],
        [ 0.0678,  0.1222],
        [ 0.3568,  0.2836],
        [ 0.7428, -0.0625],
        [-0.2052,  0.1990],
        [-0.3254, -0.0954],
        [ 0.4623,  0.5187],
        [-0.6425,  0.1566],
        [-1.2519, -0.3031],
        [-0.4688,  0.7244]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
torch.max(outputs,1)[0]

tensor([ 1.2452,  0.1660,  0.0464,  0.7897,  0.5860,  0.1222,  0.3568,  0.7428,
         0.1990, -0.0954,  0.5187,  0.1566, -0.3031,  0.7244], device='cuda:0',
       grad_fn=<MaxBackward0>)

In [ ]:
torch.max(outputs,1)[1]

tensor([1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [ ]:
preds = torch.max(outputs, 1)[1]

In [ ]:
preds

tensor([1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [ ]:
labels

tensor([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1], device='cuda:0')

In [ ]:
preds == labels

tensor([False, False, False,  True,  True,  True, False,  True, False,  True,
         True,  True,  True,  True], device='cuda:0')

In [ ]:
labels.shape[0]

14

In [ ]:
torch.sum(preds == labels)/labels.shape[0]

tensor(0.6429, device='cuda:0')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# Train the AlexNet model
alexnet.train()

num_epochs = 20

for epoch in range(num_epochs):
    print(epoch, " of ", num_epochs - 1)
    print('-' * 10)

    running_corrects = 0

    for inputs, labels in dataloaders["train"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        

        outputs = alexnet(inputs)
        preds = torch.max(outputs, 1)[1]



        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_corrects += torch.sum(preds == labels.data)

    print('Train Acc: {:.4f}'.format(running_corrects / dataset_sizes["train"]))




    # Evaluate the AlexNet model on Validation Data
    alexnet.eval()

    running_corrects = 0

    for inputs, labels in dataloaders["valid"]:
        inputs = inputs.to(device)
        labels = labels.to(device)


        outputs = alexnet(inputs)
        preds = torch.max(outputs, 1)[1]


        running_corrects += torch.sum(preds == labels.data)

    acc_valid = running_corrects / dataset_sizes["valid"]
    print('Valid Acc: {:.4f}'.format(acc_valid))
    if acc_valid > 0.99:
        print("Done!")
        break


print('Training complete')

0  of  19
----------
Train Acc: 0.5000
Valid Acc: 0.7000
1  of  19
----------
Train Acc: 0.7000
Valid Acc: 0.7000
2  of  19
----------
Train Acc: 0.9000
Valid Acc: 0.9000
3  of  19
----------
Train Acc: 0.9000
Valid Acc: 0.9000
4  of  19
----------
Train Acc: 0.9667
Valid Acc: 0.9000
5  of  19
----------
Train Acc: 0.9667
Valid Acc: 1.0000
Done!
Training complete


In [ ]:
pwd

'/content/gdrive/MyDrive/AI_Test_Kim_Project/Lil_Kim_Test'

In [ ]:
model_path = '/content/gdrive/MyDrive/AI_Test_Kim_Project/Lil_Kim_Test/alexnet_classification.pth'

In [ ]:
# Save the trained model
torch.save(alexnet.state_dict(), model_path)

In [ ]:
ls

alexnet_classification.pth  train/  valid/


In [ ]:
import requests
from PIL import Image
from io import BytesIO

# 1. Load the trained AlexNet model
def load_model(model_path):
    model = models.alexnet()
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, len(class_names))
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model.to(device)

# 2. Define a function to load an image from a URL and preprocess it
def preprocess_image(url, transform):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img_tensor = transform(img)
    return img_tensor.unsqueeze(0).to(device)

# 3. Perform inference using the loaded model
def predict_image_url(url, model):
    img_tensor = preprocess_image(url, data_transforms['valid'])
    output = model(img_tensor)
    pred = torch.max(output, 1)[1]
    return class_names[pred]

In [ ]:
trained_model = load_model(model_path)

In [ ]:
image_url = 'https://assets1.cbsnewsstatic.com/hub/i/2011/12/18/ec879496-a643-11e2-a3f0-029118418759/kim_jong_il_52010753.jpg'

In [ ]:
prediction = predict_image_url(image_url, trained_model)

In [ ]:
print("The predicted class for the input image is:", prediction)

The predicted class for the input image is: Kim Jong UN
